In [1]:
import os
import pandas as pd

In [2]:
dataset_dir = 'exports/shock_datasets'
datasets = []

for filename in os.listdir(dataset_dir):
    if filename.endswith('gzip'):
        df = pd.read_parquet(dataset_dir + '/' + filename)
        datasets.append(df.reset_index())
    else:
        continue
        
df = pd.concat(datasets, ignore_index=True)

In [3]:
kp_range = sorted(df.kp.unique())
ki_range = sorted(df.ki.unique())

print(f'''
Range of Kp values: {kp_range}
Range of Ki values: {ki_range}
''')


Range of Kp values: [-0.0001, -9e-05, -8e-05, -7.000000000000001e-05, -6.000000000000001e-05, -5e-05, -4e-05, -3.0000000000000004e-05, -2e-05, -1e-05, -9e-06, -8e-06, -7e-06, -6e-06, -4.9999999999999996e-06, -4e-06, -3e-06, -2e-06, -1e-06, -9e-07, -8e-07, -7e-07, -6e-07, -5e-07, -4e-07, -3e-07, -2e-07, -1e-07]
Range of Ki values: [0.0]



In [4]:
metric_datasets = []

for kp in df.kp.unique():
    for subset in [1, 4, 7]:
        dataset = df[(df.subset == subset) & (df.kp == kp)]
        if dataset.shape[0] == 0:
            continue
        reasonable_steady_state_error = 0.01
        dataset["pct_error"] = dataset["error_hat"] / dataset["debt_price"]
        dataset["pct_error"] = dataset["pct_error"].abs()
        recovery_time = (
            dataset[dataset.pct_error > reasonable_steady_state_error]
            .groupby(["run"])["timedelta"]
            .sum()
            .mean()
            / 3600
        )
        
        try:
            pv_initial = dataset['debt_price'].iloc[0]
            pv_max = dataset['debt_price'].max()

            target_initial = dataset[dataset.target_price > pv_initial].iloc[0]
            time_constant = dataset[dataset.target_price - pv_initial >= 0.63 * (pv_max - pv_initial)].iloc[0].timestamp - target_initial.timestamp
            time_constant = time_constant / pd.Timedelta(hours=1)
        except IndexError:
            time_constant = -1
        
        metric_datasets.append(dataset.assign(**{
            'recovery_time': recovery_time,
            'time_constant': time_constant
        }))

<ipython-input-4-3eaff2f448d3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["pct_error"] = dataset["error_hat"] / dataset["debt_price"]
<ipython-input-4-3eaff2f448d3>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["pct_error"] = dataset["pct_error"].abs()


In [5]:
metric_df = pd.concat(metric_datasets, ignore_index=True)
metric_df

,level_0,index,error_star,error_hat,old_error_star,old_error_hat,error_star_integral,error_hat_integral,error_star_derivative,error_hat_derivative,...,simulation,subset,run,substep,timestep,kp,ki,pct_error,recovery_time,time_constant
0,2884,17284,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,...,1,1,1,0,0,-0.000003,0.0,0.000000,119.081250,91.599444
1,2885,17290,0.002300,0.002300,0.0,0.0,6.175500,6.175500,4.283054e-07,4.283054e-07,...,1,1,1,6,1,-0.000003,0.0,0.002300,119.081250,91.599444
2,2886,17296,0.003421,0.003459,0.0,0.0,16.979526,17.226129,2.922051e-07,2.922051e-07,...,1,1,1,6,2,-0.000003,0.0,0.003459,119.081250,91.599444
3,2887,17302,0.003991,0.004068,0.0,0.0,30.084605,31.526380,1.499958e-07,1.499958e-07,...,1,1,1,6,3,-0.000003,0.0,0.004068,119.081250,91.599444
4,2888,17308,0.004279,0.004401,0.0,0.0,52.347745,54.410170,5.327201e-08,5.327201e-08,...,1,1,1,6,4,-0.000003,0.0,0.004401,119.081250,91.599444
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242251,23067,138247,0.000108,-0.000055,0.0,0.0,-8892.149300,453.391109,6.514412e-08,6.514412e-08,...,7,7,4,6,716,-0.000050,0.0,0.000055,365.155208,39.486667
242252,23068,138253,0.000586,0.000447,0.0,0.0,-8890.187399,454.416990,9.160660e-08,9.160660e-08,...,7,7,4,6,717,-0.000050,0.0,0.000447,365.155208,39.486667
242253,23069,138259,0.000827,0.000841,0.0,0.0,-8886.079039,457.992239,4.342518e-08,4.342518e-08,...,7,7,4,6,718,-0.000050,0.0,0.000841,365.155208,39.486667
242254,23070,138265,0.000799,0.001043,0.0,0.0,-8881.408068,463.312546,-5.020000e-09,-5.020000e-09,...,7,7,4,6,719,-0.000050,0.0,0.001043,365.155208,39.486667


In [ ]:
metric_datasets = []

for kp in metric_df.kp.unique():
    df = metric_df[(metric_df.kp == kp) & (metric_df.time_constant > -1)]
    mean_time_constant = df.time_constant.mean()
    metric_datasets.append(metric_df.assign(**{'mean_time_constant': mean_time_constant}))
    
metric_df = pd.concat(metric_datasets, ignore_index=True)
metric_df

In [ ]:
df_mean_time_constant = metric_df.query(f'time_constant >= {24*3} and time_constant <= {24*11}')

for kp in sorted(df_mean_time_constant.kp.unique()):
    df = df_mean_time_constant.query(f'kp == {kp}')
    print(f'''
    Ideal Kp range: {kp}
    Mean time constant: {df.mean_time_constant}
    ''')

In [ ]:
df_mean_time_constant.kp.unique()